<a href="https://colab.research.google.com/github/adnan-mujagic/steam-recommender-system-using-implicitly-inferred-ratings/blob/main/recommender_system_using_surprise_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Installing surprise library and importing necessary dependencies

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626466 sha256=1538ae324dfa8c2940200b6d2e56ddcf29998e7702d40f606bb96cdce3fcd47b
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [24]:
from surprise import accuracy, Dataset, Reader, KNNBasic, KNNWithMeans, KNNWithZScore, SVD
from surprise.model_selection import train_test_split

import numpy as np
import pandas as pd

## Importing the data

In [17]:
# Load the movielens-100k dataset (download it if needed),
data = pd.read_csv("/content/drive/MyDrive/ML/Steam/steam-200k-with-ratings.csv")

data.sample(5)

,user_id,game_name,rating
115232,58345543,Cubic Castles,1.344707
113344,214658749,March of War,1.344707
87264,95084569,Hitman Sniper Challenge,1.344707
113190,77214425,Red Stone Online,4.762871
31782,11978743,RAW - Realms of Ancient War,1.344707


Convert dataframe into a surprise `Dataset`

In [18]:
reader = Reader(rating_scale=(0, 5))

In [19]:
dataset = Dataset.load_from_df(data[["user_id", "game_name", "rating"]], reader)

## Splitting the data
Generating a training dataset and test dataset

In [20]:
trainset, testset = train_test_split(dataset, test_size=0.25)

Instantiating the recommender algorithm

In [22]:
# Using a basic collaborative filtering algorithm KNNWithMeans - takes into account the mean rating of each user
algo = KNNWithMeans()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


Now generate predictions

In [23]:
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0541


1.0541494444748554

Let's try some other algorithms and find the one with the lowest error

In [51]:
def rank(recommender_algorithms, by = "rmse"):
  results = pd.DataFrame(columns = ["algorithm", "rmse", "mae"])

  for algorithm in recommender_algorithms:
    algo = algorithm()

    algo.fit(trainset)

    predictions = algo.test(testset)

    row = {
      "algorithm": type(algo).__name__,
      "rmse": accuracy.rmse(predictions),
      "mae": accuracy.mae(predictions)
    }
    
    results = results.append(row, ignore_index = True)
    
  return results.sort_values(by)


In [52]:
possible_algos = [KNNBasic, KNNWithMeans, KNNWithZScore, SVD]

rank(possible_algos)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0662
MAE:  0.7162
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0541
MAE:  0.7532
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0655
MAE:  0.7517
RMSE: 1.0176
MAE:  0.7208


,algorithm,rmse,mae
3,SVD,1.017628,0.720834
1,KNNWithMeans,1.054149,0.753186
2,KNNWithZScore,1.065468,0.751690
0,KNNBasic,1.066165,0.716218
